In [1]:
import sys
import os
sys.path.append(os.getcwd())
import init


STBX not loaded: No module named 'pandas'
init loaded


In [2]:
if init.ENVIRONMENT == 'blaze':
    import subprocess

    command = 'source /usr/local/cuda/CUDA_VISIBILITY.csh'
    process = subprocess.Popen(command, shell=True, executable="/bin/csh", stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    os.environ['CUDA_VISIBLE_DEVICES'] = stdout.decode()[-2]
    # os.environ['CUDA_HOME'] = '/opt/cuda/cuda-10.0'
    print(stdout.decode())

    command = 'source /server/opt/cuda/enable_cuda_11.0'
    process = subprocess.Popen(command, shell=True, executable="/bin/csh", stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    command = 'echo $CUDA_VISIBLE_DEVICES'
    process = subprocess.Popen(command, shell=True, executable="/bin/csh", stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    print(command)
    print(stdout.decode())


GPU 0 is in use
GPU 2 is free.
Setting environment to use GPU: 1

echo $CUDA_VISIBLE_DEVICES
1



In [3]:
import tensorflow as tf
import time
import importlib

from dataset.reader import Reader
import sis_toolbox as tbx


In [4]:
MODEL_NAME = 'pix2pix'
BATCH_SIZE = 1
SUFFIX = 'test'
SHUFFLE = 'n'
PROGRESS_FREQ = 1000
SAVE_FREQ = 5000

SUBFOLDER = f'{init.TIMESTAMP}_{MODEL_NAME}_{BATCH_SIZE}x{init.TILESIZE}'
if SUFFIX is not None:
    SUBFOLDER += f'_{SUFFIX}'


In [5]:
path_logs = os.path.join(init.OUTPUT_ROOT, f'{SUBFOLDER}/logs/')
path_ckpt = os.path.join(init.OUTPUT_ROOT, f'{SUBFOLDER}/ckpt/')
path_imgs = os.path.join(init.OUTPUT_ROOT, f'{SUBFOLDER}/samples/')
path_model = os.path.join(init.OUTPUT_ROOT, f'{SUBFOLDER}/model/')
os.makedirs(path_logs, exist_ok=True)
os.makedirs(path_ckpt, exist_ok=True)
os.makedirs(path_imgs, exist_ok=True)
os.makedirs(path_model, exist_ok=True)


In [6]:
# Dynamically import all classes in the directory
directory = 'model'
modules = []
for filename in os.listdir(directory):
    if filename.endswith('.py'):
        module_name = filename[:-3]
        modules.append(importlib.import_module("." + module_name, package=directory))

model = None
for module in modules:
    if module.__name__[-len(MODEL_NAME):] == MODEL_NAME:
        model = module.GAN(path_logs, path_ckpt)


In [7]:
generator = model.generator
discriminator = model.discriminator


In [9]:
tf.keras.utils.plot_model(generator, show_shapes=True, expand_nested=False, to_file=os.path.join(path_model, 'generator.png'))
tf.keras.utils.plot_model(discriminator, show_shapes=True, expand_nested=False, to_file=os.path.join(path_model, 'discriminator.png'))
# tf.keras.utils.plot_model(generator, show_shapes=True, expand_nested=False, show_layer_activations=True, to_file=os.path.join(path_model, 'generator.png'))
# tf.keras.utils.plot_model(discriminator, show_shapes=True, expand_nested=False, show_layer_activations=True, to_file=os.path.join(path_model, 'discriminator.png'))
None


In [8]:
shuffle = False if SHUFFLE == 'n' else True
dataset_reader = Reader(BATCH_SIZE, shuffle, 'train.py', (25000, 5000))
train_dataset = dataset_reader.train_dataset
test_dataset = dataset_reader.test_dataset


datamodel.Reader called by train.py
selected random sample train: 25000
selected random sample val: 25000


In [9]:
def fit(train_ds, test_ds, steps):
    # example_target, example_input = next(iter(test_ds.take(1)))
    start = time.time()
    example_targets = []
    example_inputs = []
    for example_target, example_input in test_dataset.take(5):
        example_targets.append(example_target[0])
        example_inputs.append(example_input[0])
    example_inputs = tf.stack(example_inputs, axis=0)
    example_targets = tf.stack(example_targets, axis=0)

    for step, (target, input_image) in train_ds.repeat().take(steps).enumerate():
        if step % PROGRESS_FREQ == 0:
            # display.clear_output(wait=True)
            
            if step != 0:
                print(f'Time taken for {PROGRESS_FREQ} steps: {time.time()-start:.2f} sec\n')
                start = time.time()
            
            tbx.generate_images(generator, example_inputs, example_targets, showimg=False, PATH_IMGS=path_imgs, savemodel=model.name, starttimestamp=init.TIMESTAMP, iteration=step)
            # for example_target, example_input in test_dataset.take(1):
            #     helper.generate_images(generator, example_input, example_target, showimg=False, PATH_IMGS=path_imgs, savemodel=model.name, starttimestamp=STARTTIME, iteration=step)

            print(f"Step: {step}")

        model.train_step(input_image, target, step)

        # Training step
        if (step + 1) % 10 == 0:
            print('.', end='', flush=True)

        # Save (checkpoint) the model every 5k steps
        if (step + 1) % SAVE_FREQ == 0:
            print(f'Step + 1 = {step + 1} - saving checkpoint')
            model.save()


In [10]:
fit(train_dataset, test_dataset, steps=40000)


2023-07-12 16:02:22.079295: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


Step: 0
....................................................................................................Time taken for 1000 steps: 373.68 sec

Step: 1000
....................................................................................................Time taken for 1000 steps: 367.22 sec

Step: 2000
....................................................................................................Time taken for 1000 steps: 366.45 sec

Step: 3000
....................................................................................................Time taken for 1000 steps: 369.51 sec

Step: 4000
....................................................................................................Step + 1 = 5000 - saving checkpoint
Time taken for 1000 steps: 378.63 sec

Step: 5000
................................

KeyboardInterrupt: 

In [ ]:
import random

sample_dataset = tf.data.TFRecordDataset(os.path.join(init.VAL_DIR, random.choice(os.listdir(init.VAL_DIR))))
for element in sample_dataset:
    tbx.plot_tensor_sbs(element, init.TILESIZE)
    s2_tensor, s3_tensor = tbx.parse_tfrecord(element, init.TILESIZE)
    s2_tensor, s3_tensor = dataset_reader.resize(s2_tensor, s3_tensor, init.IMG_HEIGHT, init.IMG_WIDTH)



In [ ]:
from sis_toolbox import RGBProfile as rgb

gen_output = generator(s3_tensor[tf.newaxis, ...], training=False)
tbx.plot_tensor(gen_output[0], rgb.S2)


In [ ]:
import matplotlib.pyplot as plt

disc_out = discriminator([s3_tensor[tf.newaxis, ...], gen_output], training=False)
plt.imshow(disc_out[0, ..., -1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()


In [ ]:
example_targets = []
example_inputs = []
for example_target, example_input in test_dataset.take(5):
    example_targets.append(example_target[0])
    example_inputs.append(example_input[0])

tbx.generate_images(generator, tf.stack(example_inputs, axis=0), tf.stack(example_targets, axis=0))
